## 기초 실습) 사이킷런을 이용하여 붓꽃(Iris) 데이터 품종 예측하기



In [ ]:
# 사이킷런 버전 확인
import sklearn
print(sklearn.__version__)

1.2.2


**붓꽃 예측을 위한 사이킷런 필요 모듈 로딩**

In [ ]:
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split

**데이터 세트를 로딩**

In [ ]:
import pandas as pd

# 붓꽃 데이터 세트를 로딩합니다.
iris = load_iris()

# iris.data는 Iris 데이터 세트에서 피처(feature)만으로 된 데이터를 numpy로 가지고 있습니다.
iris_data = iris.data

# iris.target은 붓꽃 데이터 세트에서 레이블(결정 값) 데이터를 numpy로 가지고 있습니다.
iris_label = iris.target
print('iris target값:', iris_label)
print('iris target명:', iris.target_names)

# 붓꽃 데이터 세트를 자세히 보기 위해 DataFrame으로 변환합니다.
iris_df = pd.DataFrame(data=iris_data, columns=iris.feature_names)
iris_df['label'] = iris.target
iris_df.head(3)

iris target값: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2]
iris target명: ['setosa' 'versicolor' 'virginica']


,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),label
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0


**학습 데이터와 테스트 데이터 세트로 분리**
- train_test_split()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(iris_data, iris_label,
                                                    test_size=0.2, random_state=11)

**학습 데이터 세트로 학습(Train)수행**

In [ ]:
# DecisionTreeClassifier 객체 생성
dt_clf = DecisionTreeClassifier(random_state=11)

# 학습 수행
dt_clf.fit(X_train, y_train)

DecisionTreeClassifier(random_state=11)

**테스트 데이터 세트로 예측(Predict) 수행**

In [ ]:
# 학습이 완료된 DecisionTreeClassifier 객체에서 테스트 데이터 세트로 예측 수행.
pred = dt_clf.predict(X_test)

In [ ]:
pred

array([2, 2, 1, 1, 2, 0, 1, 0, 0, 1, 1, 1, 1, 2, 2, 0, 2, 1, 2, 2, 1, 0,
       0, 1, 0, 0, 2, 1, 0, 1])

**예측 정확도 평가**

In [ ]:
from sklearn.metrics import accuracy_score
print('예측 정확도: {0:.4f}'.format(accuracy_score(y_test,pred)))

예측 정확도: 0.9333


In [4]:
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

iris = load_iris()
dt_clf = DecisionTreeClassifier()
train_data = iris.data
train_target = iris.target
dt_clf.fit(train_data, train_target)

# 학습 데이터 셋으로 예측 수행
pred = dt_clf.predict(train_data)
print('예측 정확도:',accuracy_score(train_target,pred))

예측 정확도: 1.0


## 1. 교차 검증
- (1) KFold 검증
- (2) Strified 검증
- (3) cross_val_score(): 교차 검증을 쉽게 도와주는 함수

### (1) K 폴드

In [5]:
import numpy as np
from sklearn.model_selection import KFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

# 모델 학습 및 평가 함수 정의
def train_and_evaluate(train_index, test_index, model, train_data, train_target):
    # 학습 및 검증 데이터 분리
    X_train, X_test = train_data[train_index], train_data[test_index]
    y_train, y_test = train_target[train_index], train_target[test_index]

    # 모델 학습 및 예측
    model.fit(X_train, y_train)
    predictions = model.predict(X_test)

    # 정확도 계산
    accuracy = accuracy_score(y_test, predictions)
    return accuracy, X_train.shape[0], X_test.shape[0], test_index

# K-Fold 교차 검증 설정
kfold = KFold(n_splits=5)
cv_accuracy = []
dt_clf = DecisionTreeClassifier()

# K-Fold 교차 검증 수행
for fold, (train_index, test_index) in enumerate(kfold.split(train_data), 1):
    accuracy, train_size, test_size, test_idx = train_and_evaluate(
        train_index, test_index, dt_clf, train_data, train_target
    )
    cv_accuracy.append(accuracy)

    print(f"{fold}번째 트레이닝")
    print(f"검증 정확도: {accuracy}, 학습 데이터 크기: {train_size}, 검증 데이터 크기: {test_size}")
    print(f"검증 데이터 셋 인덱스 값: {test_idx}")
    print("===" * 10)

# 최종 정확도 평균 출력
print("최종 정확도 평균 값:", np.mean(cv_accuracy))

1번째 트레이닝
검증 정확도: 1.0, 학습 데이터 크기: 120, 검증 데이터 크기: 30
검증 데이터 셋 인덱스 값: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29]
2번째 트레이닝
검증 정확도: 0.9666666666666667, 학습 데이터 크기: 120, 검증 데이터 크기: 30
검증 데이터 셋 인덱스 값: [30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53
 54 55 56 57 58 59]
3번째 트레이닝
검증 정확도: 0.8666666666666667, 학습 데이터 크기: 120, 검증 데이터 크기: 30
검증 데이터 셋 인덱스 값: [60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83
 84 85 86 87 88 89]
4번째 트레이닝
검증 정확도: 0.9333333333333333, 학습 데이터 크기: 120, 검증 데이터 크기: 30
검증 데이터 셋 인덱스 값: [ 90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119]
5번째 트레이닝
검증 정확도: 0.7333333333333333, 학습 데이터 크기: 120, 검증 데이터 크기: 30
검증 데이터 셋 인덱스 값: [120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137
 138 139 140 141 142 143 144 145 146 147 148 149]
최종 정확도 평균 값: 0.9


### (2) Stratified K 폴드

In [9]:
import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import StratifiedKFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

# Iris 데이터셋 로드
iris = load_iris()
features = iris.data
label = iris.target

# 모델 초기화
dt_clf = DecisionTreeClassifier(random_state=156)

# 모델 학습 및 평가 함수 정의
def train_and_evaluate(train_index, test_index, model, features, label):
    # 학습 및 검증 데이터 분리
    X_train, X_test = features[train_index], features[test_index]
    y_train, y_test = label[train_index], label[test_index]

    # 모델 학습 및 예측
    model.fit(X_train, y_train)
    predictions = model.predict(X_test)

    # 정확도 계산
    accuracy = np.round(accuracy_score(y_test, predictions), 4)
    return accuracy, X_train.shape[0], X_test.shape[0], test_index

# Stratified K-Fold 교차 검증 설정
skfold = StratifiedKFold(n_splits=3)
cv_accuracy = []

# Stratified K-Fold 교차 검증 수행
for fold, (train_index, test_index) in enumerate(skfold.split(features, label), 1):
    accuracy, train_size, test_size, test_idx = train_and_evaluate(
        train_index, test_index, dt_clf, features, label
    )
    cv_accuracy.append(accuracy)

    # 출력 메시지 변경
    print(f"{fold}번째 트레이닝")
    print(f"검증 정확도: {accuracy}, 학습 데이터 크기: {train_size}, 검증 데이터 크기: {test_size}")
    print(f"검증 데이터 셋 인덱스 값: {test_idx}")
    print("===" * 10)

# 교차 검증별 정확도 및 평균 정확도 계산
print("\n최종 정확도 평균 값:", np.mean(cv_accuracy))

1번째 트레이닝
검증 정확도: 0.98, 학습 데이터 크기: 100, 검증 데이터 크기: 50
검증 데이터 셋 인덱스 값: [  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  50
  51  52  53  54  55  56  57  58  59  60  61  62  63  64  65  66 100 101
 102 103 104 105 106 107 108 109 110 111 112 113 114 115]
2번째 트레이닝
검증 정확도: 0.94, 학습 데이터 크기: 100, 검증 데이터 크기: 50
검증 데이터 셋 인덱스 값: [ 17  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  67
  68  69  70  71  72  73  74  75  76  77  78  79  80  81  82 116 117 118
 119 120 121 122 123 124 125 126 127 128 129 130 131 132]
3번째 트레이닝
검증 정확도: 0.98, 학습 데이터 크기: 100, 검증 데이터 크기: 50
검증 데이터 셋 인덱스 값: [ 34  35  36  37  38  39  40  41  42  43  44  45  46  47  48  49  83  84
  85  86  87  88  89  90  91  92  93  94  95  96  97  98  99 133 134 135
 136 137 138 139 140 141 142 143 144 145 146 147 148 149]

최종 정확도 평균 값: 0.9666666666666667


### (3) cross_val_score() : 교차 검증을 쉽게 도와주는 함수

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score
from sklearn.datasets import load_iris
import numpy as np

iris_data = load_iris()
dt_clf = DecisionTreeClassifier(random_state=156)

data = iris_data.data # x
label = iris_data.target # y

scores = cross_val_score(dt_clf , data , label , scoring='accuracy',cv=3) # scoring=평가지표, cv=교차 검증 폴드 수

#print(scores, type(scores))
print('교차 검증별 정확도:',np.round(scores, 4))
print('평균 검증 정확도:', np.round(np.mean(scores), 4))

교차 검증별 정확도: [0.98 0.94 0.98]
평균 검증 정확도: 0.9667


- cross_val_predict()
  - cross_val_predict 함수를 사용하여 교차 검증을 수행하고 예측값을 얻을 수 있다.

In [ ]:
from sklearn.model_selection import cross_val_predict

predict = cross_val_predict(dt_clf , data , label ,cv=3) # cv=교차 검증 폴드 수

print('교차별 예측 값:', predict)

교차별 예측 값: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 1 1 1
 1 1 1 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 1 2 2 2 2
 2 2 2 2 2 2 2 2 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 1 2 2 2 2 2 2 2 2 2
 2 2]


### GridSearchCV
- Decision Tree 모델의 하이퍼파라미터를 튜닝

In [ ]:
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import accuracy_score

# 데이터를 로딩하고 학습데이타와 테스트 데이터 분리
iris = load_iris()
X_train, X_test, y_train, y_test = train_test_split(iris_data.data, iris_data.target,
                                                    test_size=0.2, random_state=121)
dtree = DecisionTreeClassifier()

### parameter 들을 dictionary 형태로 설정
# 'max_depth': [1, 2, 3]: Decision Tree의 최대 깊이를 1, 2, 3으로 테스트합니다.
# 'min_samples_split': [2, 3]: 노드를 분할하기 위한 최소 샘플 개수를 2와 3으로 테스트합니다.

parameters = {'max_depth':[1, 2, 3], 'min_samples_split':[2,3]}

In [ ]:
import pandas as pd

# param_grid의 하이퍼 파라미터들을 3개의 train, test set fold 로 나누어서 테스트 수행 설정.
### refit=True 가 default 임. True이면 가장 좋은 파라미터 설정으로 재 학습 시킴.

# 'cv=3': 교차 검증(cross-validation)을 3-fold로 수행합니다.
# 'refit=True': 최적의 하이퍼파라미터 조합을 찾은 후, 해당 조합으로 전체 학습을 수행하여 모델을 최종적으로 구성합니다.
#' return_train_score=True': 학습 데이터에 대한 성능 점수도 반환하도록 설정합니다.

grid_dtree = GridSearchCV(dtree, param_grid=parameters, cv=3, refit=True, return_train_score=True)

# 붓꽃 Train 데이터로 param_grid의 하이퍼 파라미터들을 순차적으로 학습/평가 .
grid_dtree.fit(X_train, y_train)

# GridSearchCV 결과는 cv_results_ 라는 딕셔너리로 저장됨. 이를 DataFrame으로 변환
scores_df = pd.DataFrame(grid_dtree.cv_results_)
scores_df[['params', 'mean_test_score', 'rank_test_score',
           'split0_test_score', 'split1_test_score', 'split2_test_score']]

In [ ]:
grid_dtree.cv_results_

In [ ]:
print('GridSearchCV 최적 파라미터:', grid_dtree.best_params_)
print('GridSearchCV 최고 정확도: {0:.4f}'.format(grid_dtree.best_score_))

# refit=True로 설정된 GridSearchCV 객체가 fit()을 수행 시 학습이 완료된 Estimator를 내포하고 있으므로 predict()를 통해 예측도 가능.
pred = grid_dtree.predict(X_test)
print('테스트 데이터 세트 정확도: {0:.4f}'.format(accuracy_score(y_test,pred)))

In [ ]:
# GridSearchCV의 refit으로 이미 학습이 된 estimator 반환
estimator = grid_dtree.best_estimator_

# GridSearchCV의 best_estimator_는 이미 최적 하이퍼 파라미터로 학습이 됨
pred = estimator.predict(X_test)
print('테스트 데이터 세트 정확도: {0:.4f}'.format(accuracy_score(y_test,pred)))

## 데이터 인코딩
- (1) Label Encoding
- (2) OneHot Encoding : Scikit-Learn 라이브러리에서 제공하는 기능
- (3) GetDummies : Pandas에서 제공하는 기능

**레이블 인코딩(Label encoding)**
레이블 인코딩(Label Encoding)은 범주형 데이터를 숫자형 데이터로 변환하는 방법 중 하나입니다. 이 기법은 범주형 레이블(예: 'red', 'blue', 'green' 등)을 숫자 값으로 변환하여 머신러닝 알고리즘이 이해할 수 있도록 만듭니다.

In [ ]:
from sklearn.preprocessing import LabelEncoder

items=['강남구','서초구','송파구','노원구','마포구','마포구','용산구','용산구']

# LabelEncoder를 객체로 생성한 후 , fit( ) 과 transform( ) 으로 label 인코딩 수행.
encoder = LabelEncoder()
encoder.fit(items)
labels = encoder.transform(items)
print('인코딩 변환값:',labels) # 문자열 값 -> 숫자형 값으로 변경 (Label Encoding)

In [ ]:
print('인코딩 클래스:',encoder.classes_)

In [ ]:
# 숫자 -> 문자열 값으로 변경
print('디코딩 원본 값:',encoder.inverse_transform([4, 5, 2, 0, 1, 1, 3, 3]))

* 원-핫 인코딩(One-Hot encoding)
- 레이블 인코딩의 단점을 보완하기 위해 나옴
(용산구의 값이 5라는 것과 강남구가 0이라는 것은 아무 상관이 없는데 알고리즘에 따라서는 영향을 받을 수 있다.)
- 원-핫: 하나만 핫하다!

In [ ]:
from sklearn.preprocessing import OneHotEncoder
import numpy as np

items=['강남구','서초구','송파구','노원구','마포구','마포구','용산구','용산구']

# 2차원 ndarray로 변환합니다.
# reshape() 함수의 첫 번째 인자로 -1을 주면 해당 차원의 크기를 자동으로 계산하여 맞춰줍니다.
# 두 번째 인자로 1을 주면 열의 크기를 1로 지정하여 결과적으로 1개의 열로 이루어진 2차원 배열을 생성합니다.
# 이렇게 2차원 배열로 변환된 데이터는 머신 러닝 모델에 입력으로 사용하기에 적합한 형태가 됩니다.
# 머신 러닝 모델은 입력으로 2차원 배열을 기대하므로, 데이터를 이와 같은 형태로 변환하여 모델에 사용해야 합니다.
items = np.array(items).reshape(-1, 1)

# 원-핫 인코딩을 적용합니다.
oh_encoder = OneHotEncoder()
oh_encoder.fit(items)
oh_labels = oh_encoder.transform(items)

# OneHotEncoder로 변환한 결과는 희소행렬(Sparse Matrix)이므로 toarray()를 이용하여 밀집 행렬(Dense Matrix)로 변환.
print('원-핫 인코딩 데이터')
print(oh_labels.toarray())
print('원-핫 인코딩 데이터 차원')
print(oh_labels.shape)

In [ ]:
import pandas as pd

df = pd.DataFrame({'item':['강남구','서초구','송파구','노원구','마포구','마포구','용산구','용산구']})
df

In [ ]:
pd.get_dummies(df)

In [ ]:
import pandas as pd

data = {'Fruit': ['Apple', 'Banana', 'Cherry', 'Apple', 'Cherry'],
        'Color': ['Red', 'Yellow', 'Red', 'Green', 'Red']}
df = pd.DataFrame(data)
df

In [ ]:
# columns: 이 매개변수는 원-핫 인코딩을 적용하려는 열(특성)을 지정합니다.
# 여기에 나열된 열만 원-핫 인코딩이 적용됩니다.
# 나머지 열은 그대로 남습니다. 리스트나 배열로 여러 열을 지정할 수 있습니다.

# prefix: 이 매개변수는 원-핫 인코딩된 새로운 열의 이름을 지정합니다.
# 각 범주마다 새로운 열이 생성되며, prefix에 지정한 값을 접두사로 사용하여 열 이름을 생성합니다.
# 이를 통해 새로운 열과 원래 열을 구분할 수 있습니다.

df_encoded = pd.get_dummies(df)
df_encoded

In [ ]:
df_encoded = pd.get_dummies(df, columns=['Fruit', 'Color'], prefix=['Fruit', 'Color'])
df_encoded

## 3. 피처 스케일링과 정규화
- 데이터 전처리 기법으로, 기계 학습 알고리즘이 더 나은 성능을 발휘할 수 있도록 데이터를 변환하는 과정
- 데이터의 크기나 분포를 조정하여 모델 학습과 예측의 안정성과 정확성을 향상시키는 역할

**(1) StandardScaler**
- 데이터의 각 특성(열)을 평균이 0이고 표준 편차가 1인 표준 정규 분포로 변환할 수 있습니다.
- 데이터의 분포가 정규 분포에 가깝거나 정규화가 필요한 경우.
- 대부분의 통계 기반 모델 또는 거리 기반 알고리즘(예: K-평균 클러스터링)을 사용할 때.

In [10]:
from sklearn.datasets import load_iris
import pandas as pd
# 붓꽃 데이터 셋을 로딩하고 DataFrame으로 변환합니다.
iris = load_iris()
iris_data = iris.data
iris_df = pd.DataFrame(data=iris_data, columns=iris.feature_names)

print(iris_df)
print('feature 들의 평균 값')
print(iris_df.mean())
print('\nfeature 들의 분산 값')
print(iris_df.var())

     sepal length (cm)  sepal width (cm)  petal length (cm)  petal width (cm)
0                  5.1               3.5                1.4               0.2
1                  4.9               3.0                1.4               0.2
2                  4.7               3.2                1.3               0.2
3                  4.6               3.1                1.5               0.2
4                  5.0               3.6                1.4               0.2
..                 ...               ...                ...               ...
145                6.7               3.0                5.2               2.3
146                6.3               2.5                5.0               1.9
147                6.5               3.0                5.2               2.0
148                6.2               3.4                5.4               2.3
149                5.9               3.0                5.1               1.8

[150 rows x 4 columns]
feature 들의 평균 값
sepal length (cm)    5.8

In [11]:
from sklearn.preprocessing import StandardScaler

# Standard: 표준화 -> 각 특성들이 동일한 스케일을 가지도록 만들어준다.
# StandardScaler는 특성들을 평균이 0이고 분산이 1인 표준 정규 분포를 따르도록 변환

# StandardScaler객체 생성
scaler = StandardScaler()
# StandardScaler 로 데이터 셋 변환. fit( ) 과 transform( ) 호출.
scaler.fit(iris_df)
iris_scaled = scaler.transform(iris_df)

#transform( )시 scale 변환된 데이터 셋이 numpy ndarry로 반환되어 이를 DataFrame으로 변환
iris_df_scaled = pd.DataFrame(data=iris_scaled, columns=iris.feature_names)

print(iris_df_scaled)

print('feature 들의 평균 값')
print(iris_df_scaled.mean()) # 평균값이 0에 가까워짐 - 값: -1.69 * 10^-15
print('\nfeature 들의 분산 값')
print(iris_df_scaled.var())

     sepal length (cm)  sepal width (cm)  petal length (cm)  petal width (cm)
0            -0.900681          1.019004          -1.340227         -1.315444
1            -1.143017         -0.131979          -1.340227         -1.315444
2            -1.385353          0.328414          -1.397064         -1.315444
3            -1.506521          0.098217          -1.283389         -1.315444
4            -1.021849          1.249201          -1.340227         -1.315444
..                 ...               ...                ...               ...
145           1.038005         -0.131979           0.819596          1.448832
146           0.553333         -1.282963           0.705921          0.922303
147           0.795669         -0.131979           0.819596          1.053935
148           0.432165          0.788808           0.933271          1.448832
149           0.068662         -0.131979           0.762758          0.790671

[150 rows x 4 columns]
feature 들의 평균 값
sepal length (cm)   -1.6

**(2) MinMaxScaler**
- 주어진 데이터를 최소값과 최대값을 이용하여 0과 1 사이의 값으로 변환
- X_scaled = (X - X_min) / (X_max - X_min)

- 데이터의 분포가 일반적인 분포가 아니고, 특성 간의 범위를 동일하게 조정하려는 경우.
- 신경망(Neural Network)과 같은 모델에서 주로 사용됩니다.


In [12]:
from sklearn.preprocessing import MinMaxScaler

# MinMaxScaler객체 생성
scaler = MinMaxScaler()
# MinMaxScaler 로 데이터 셋 변환. fit() 과 transform() 호출.
scaler.fit(iris_df)
iris_scaled = scaler.transform(iris_df)

# transform()시 scale 변환된 데이터 셋이 numpy ndarry로 반환되어 이를 DataFrame으로 변환
iris_df_scaled = pd.DataFrame(data=iris_scaled, columns=iris.feature_names)
print('feature들의 최소 값')
print(iris_df_scaled.min())
print('\nfeature들의 최대 값')
print(iris_df_scaled.max())


feature들의 최소 값
sepal length (cm)    0.0
sepal width (cm)     0.0
petal length (cm)    0.0
petal width (cm)     0.0
dtype: float64

feature들의 최대 값
sepal length (cm)    1.0
sepal width (cm)     1.0
petal length (cm)    1.0
petal width (cm)     1.0
dtype: float64


In [13]:
from sklearn.preprocessing import MinMaxScaler
import numpy as np

# 학습 데이터는 0 부터 10까지, 테스트 데이터는 0 부터 5까지 값을 가지는 데이터 세트로 생성
# Scaler클래스의 fit(), transform()은 2차원 이상 데이터만 가능하므로 reshape(-1, 1)로 차원 변경
train_array = np.arange(0, 11).reshape(-1, 1)
test_array =  np.arange(0, 6).reshape(-1, 1)

In [14]:
# 최소값 0, 최대값 1로 변환하는 MinMaxScaler객체 생성
scaler = MinMaxScaler()
# fit()하게 되면 train_array 데이터의 최소값이 0, 최대값이 10으로 설정.
scaler.fit(train_array)
# 1/10 scale로 train_array 데이터 변환함. 원본 10-> 1로 변환됨.
train_scaled = scaler.transform(train_array)

print('원본 train_array 데이터:', np.round(train_array.reshape(-1), 2))
print('Scale된 train_array 데이터:', np.round(train_scaled.reshape(-1), 2))

원본 train_array 데이터: [ 0  1  2  3  4  5  6  7  8  9 10]
Scale된 train_array 데이터: [0.  0.1 0.2 0.3 0.4 0.5 0.6 0.7 0.8 0.9 1. ]


In [15]:
# 앞에서 생성한 MinMaxScaler에 test_array를 fit()하게 되면 원본 데이터의 최소값이 0, 최대값이 5으로 설정됨
scaler.fit(test_array)
# 1/5 scale로 test_array 데이터 변환함. 원본 5->1로 변환.
test_scaled = scaler.transform(test_array)
# train_array 변환 출력
print('원본 test_array 데이터:', np.round(test_array.reshape(-1), 2))
print('Scale된 test_array 데이터:', np.round(test_scaled.reshape(-1), 2))

원본 test_array 데이터: [0 1 2 3 4 5]
Scale된 test_array 데이터: [0.  0.2 0.4 0.6 0.8 1. ]


In [16]:
scaler = MinMaxScaler()
scaler.fit(train_array)
train_scaled = scaler.transform(train_array)
print('원본 train_array 데이터:', np.round(train_array.reshape(-1), 2))
print('Scale된 train_array 데이터:', np.round(train_scaled.reshape(-1), 2))

# test_array에 Scale 변환을 할 때는 반드시 fit()을 호출하지 않고 transform() 만으로 변환해야 함.
test_scaled = scaler.transform(test_array)
print('\n원본 test_array 데이터:', np.round(test_array.reshape(-1), 2))
print('Scale된 test_array 데이터:', np.round(test_scaled.reshape(-1), 2))

원본 train_array 데이터: [ 0  1  2  3  4  5  6  7  8  9 10]
Scale된 train_array 데이터: [0.  0.1 0.2 0.3 0.4 0.5 0.6 0.7 0.8 0.9 1. ]

원본 test_array 데이터: [0 1 2 3 4 5]
Scale된 test_array 데이터: [0.  0.1 0.2 0.3 0.4 0.5]
